<a href="https://colab.research.google.com/github/simulate111/Computer-Vision-and-Sensor-Fusion/blob/main/Assignment4_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Assignment 4: TKO_7096-3001 Computer Vision and Sensor Fusion


Goal: develop a RGB-Depth fusion architecture for semantic segmentation based on Fully Convolutional Network (FCN) .


<font color='red'> Deadline: 25.03.2024 at 24:00.

- Imports go here

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from google.colab import files
import zipfile

### Load the dataset and Ground-truth###

- Dataset consists of 1100 (per modality) images of road scenes. It is divided into train (600 images), test (200 images) and validation (300 images) datasets.
- Change the size of all images into 256*256.
- Converting the labels into one hot encoding
- Create a DataLoader for loading the files when training the model.

In [ ]:
# Upload the dataset from your local computer
uploaded = files.upload()

# Extract the dataset if it's in a zip file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("/content/dataset")  # Change the path as needed
        print(f"Dataset extracted to /content/dataset")

In [ ]:
# Verify extraction
extracted_files = os.listdir("/content/dataset")
print("Extracted files:", extracted_files)

In [ ]:
import glob
import os

# Base dataset directory (adjusted for nested structure)
base_dir = '/content/dataset/dataset'

# Function to get .npy file paths for a specific split (train/test/validation) and modality (rgb/depth/label)
def get_npy_paths(split, modality):
    return sorted(glob.glob(os.path.join(base_dir, split, modality, '*.npy')))

# Load paths for training, validation, and testing datasets
rgb_train_paths = get_npy_paths('train', 'rgb')
depth_train_paths = get_npy_paths('train', 'depth')
label_train_paths = get_npy_paths('train', 'label')

rgb_val_paths = get_npy_paths('validation', 'rgb')
depth_val_paths = get_npy_paths('validation', 'depth')
label_val_paths = get_npy_paths('validation', 'label')

rgb_test_paths = get_npy_paths('test', 'rgb')
depth_test_paths = get_npy_paths('test', 'depth')
label_test_paths = get_npy_paths('test', 'label')

# Quick check to confirm paths are loaded
print(f"Train RGB: {len(rgb_train_paths)}, Depth: {len(depth_train_paths)}, Labels: {len(label_train_paths)}")
print(f"Validation RGB: {len(rgb_val_paths)}, Depth: {len(depth_val_paths)}, Labels: {len(label_val_paths)}")
print(f"Test RGB: {len(rgb_test_paths)}, Depth: {len(depth_test_paths)}, Labels: {len(label_test_paths)}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
import numpy as np
import cv2

class DataLoader(Sequence):
    def __init__(self, rgb_paths, depth_paths, label_paths, batch_size, num_classes=19, target_size=(256, 256)):
        self.rgb_paths = rgb_paths
        self.depth_paths = depth_paths
        self.label_paths = label_paths
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.target_size = target_size

    def __len__(self):
        return int(np.floor(len(self.rgb_paths) / self.batch_size)) # Use floor instead of ceil


    def __getitem__(self, idx):
        batch_rgb = self.rgb_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_depth = self.depth_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.label_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        return self.__generate_data(batch_rgb, batch_depth, batch_labels)

    def __generate_data(self, batch_rgb_files, batch_depth_files, batch_label_files):
        rgb_images = np.array([cv2.resize(np.load(file), self.target_size) / 255.0 for file in batch_rgb_files])
        depth_images = np.array([cv2.resize(np.concatenate([np.load(file), np.load(file), np.load(file)], axis=-1), self.target_size) / np.max(np.load(file)) if np.max(np.load(file)) != 0 else cv2.resize(np.concatenate([np.load(file), np.load(file), np.load(file)], axis=-1), self.target_size) for file in batch_depth_files])
        labels = np.array([cv2.resize(to_categorical(np.load(file), num_classes=self.num_classes), self.target_size) for file in batch_label_files])
        labels = labels.reshape(labels.shape[0], -1, labels.shape[-1]) #add this line.

        print("RGB images shape:", rgb_images.shape)
        print("Depth images shape:", depth_images.shape)
        print("Labels shape:", labels.shape)
        return (rgb_images, depth_images), labels

# ... (rest of your code)

- Visualize the data you have prepared

In [ ]:
def visualize_samples(rgb_images, depth_images, labels, num_to_display = 3): #add num_to_display
    for i in range(min(len(rgb_images), num_to_display)): #display only up to num_to_display images.
        plt.figure(figsize=(12, 4))

        # Debugging RGB values
        print(f"RGB Image min: {rgb_images[i].min()}, max: {rgb_images[i].max()}, dtype: {rgb_images[i].dtype}")

        # RGB Image (scaling if needed)
        rgb_img = rgb_images[i]
        if rgb_img.max() <= 1.0:
            rgb_img = (rgb_img * 255).astype('uint8')
        elif rgb_img.dtype != 'uint8':
            rgb_img = rgb_img.astype('uint8')

        plt.subplot(1, 3, 1)
        plt.title("RGB Image")
        plt.imshow(rgb_img)
        plt.axis('off')

        # Depth Image (grayscale, scaled)
        depth_img = depth_images[i]
        if depth_img.max() > depth_img.min(): #prevent divide by zero.
          depth_img = (depth_img - depth_img.min()) / (depth_img.max() - depth_img.min())  # Scale to [0, 1]
        plt.subplot(1, 3, 2)
        plt.title("Depth Image")
        plt.imshow(depth_img, cmap='gray')
        plt.axis('off')

        # Label Image
        plt.subplot(1, 3, 3)
        plt.title("Label")
        plt.imshow(np.argmax(labels[i], axis=-1))
        plt.axis('off')

        plt.show()


In [ ]:
print(np.load(rgb_train_paths[0]).shape)
print(np.load(depth_train_paths[0]).shape)
print(np.load(label_train_paths[0]).shape)


In [ ]:
import numpy as np

# Assuming rgb_train_paths, depth_train_paths, and label_train_paths are already defined

depth_train_data = [np.load(path) for path in depth_train_paths]
rgb_train_data = [np.load(path) for path in rgb_train_paths]
label_train_data = [np.load(path) for path in label_train_paths]

# Modify the depth data
modified_depth_data = []
for depth_img in depth_train_data:
    modified_depth_data.append(np.concatenate([depth_img, depth_img, depth_img], axis=-1))

# Modify the RGB Data (optional, but requested by the prompt)
modified_rgb_data = []
for rgb_img in rgb_train_data:
    modified_rgb_data.append(rgb_img) #no change needed, but the prompt says to copy it.

#Verify the result
print(modified_depth_data[0].shape)
print(modified_rgb_data[0].shape)
print(label_train_data[0].shape)

# Example usage with a visualization function (if you have one):
# visualize_samples(modified_rgb_data, modified_depth_data, label_train_data)

In [ ]:
batch_size = 3  # For visualization
data_loader = DataLoader(rgb_train_paths, depth_train_paths, label_train_paths, batch_size)

# Get the first batch
(data_rgb, data_depth), data_labels = data_loader[0]

# Visualize the first batch
visualize_samples(data_rgb, data_depth, data_labels)

 Define a Fully Convolutional Network (FCN) for image segmentaion by fusing RGB and depth images. The network consists of  two sterams which each stream has following layers:

    1. Use the pretrained ResNet50 on imageNet
    2. Add two Conv layers with 128 and 256 nodes, respectively. Kernel size (3,3), stride (1,1)
    3. Top of the Conv layers, add dropout layer with 0.2
    4. Concatenate two streams.
    5. Add a transposed convolution layer (Conv2DTranspose)  with Kernel size (64,64), stride (32,32)
    6. Add a softmax activation layer
    
 You can find the model summary and structure in the PDF file.


In [ ]:
class UniqueResNet50(layers.Layer):
    def __init__(self, **kwargs):
        super(UniqueResNet50, self).__init__(**kwargs)
        self.resnet = ResNet50(weights='imagenet', include_top=False)
        for layer in self.resnet.layers:
            layer.trainable = False

    def call(self, inputs):
        return self.resnet(inputs)

def build_fcn_segmentation_model(input_shape=(256, 256, 3), num_classes=19):
    """Builds a Fully Convolutional Network (FCN) for image segmentation."""

    # Input layers
    input_rgb = layers.Input(shape=input_shape, name="input_rgb")
    input_depth = layers.Input(shape=input_shape, name="input_depth")

    # ResNet50 base models (pretrained) with unique instances
    rgb_stream = UniqueResNet50()(input_rgb)
    depth_stream = UniqueResNet50()(input_depth)

    # RGB stream
    rgb_stream = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(rgb_stream)
    rgb_stream = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(rgb_stream)
    rgb_stream = layers.Dropout(0.2)(rgb_stream)

    # Depth stream
    depth_stream = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(depth_stream)
    depth_stream = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(depth_stream)
    depth_stream = layers.Dropout(0.2)(depth_stream)

    # Concatenate streams
    concatenated = layers.Concatenate()([rgb_stream, depth_stream])

    # Transposed convolution (upsampling)
    upsampled = layers.Conv2DTranspose(num_classes, (64, 64), strides=(32, 32), padding='same')(concatenated)

    # Reshape (optional, for compatibility with some loss functions)
    reshaped = layers.Reshape((-1, num_classes))(upsampled)

    # Softmax activation
    output = layers.Activation('softmax')(reshaped)

    # Create the model
    model = Model(inputs=[input_rgb, input_depth], outputs=output)
    return model

In [ ]:
# Build the model
model = build_fcn_segmentation_model()

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

# Assuming 'model' is your Keras model object
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

Compile the model with SGD(learning_rate=0.01, decay=1e-5, momentum=0.9) and loss="categorical_crossentropy"

Train the model on the “train” dataset  and “validation”dataset for epochs =10.

In [ ]:
# Compile the model
optimizer = optimizers.SGD(learning_rate=0.01, decay=1e-5, momentum=0.9)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])


In [ ]:
# Create data loaders
batch_size = 16
train_loader = DataLoader(rgb_train_paths, depth_train_paths, label_train_paths, batch_size)
val_loader = DataLoader(rgb_val_paths, depth_val_paths, label_val_paths, batch_size)


In [ ]:
# Generator function to iterate through DataLoader
def generator(data_loader):
  for idx in range(len(data_loader)):
    yield data_loader.__getitem__(idx)

In [ ]:
# Convert the DataLoaders to TensorFlow Datasets with explicit types and shapes
train_dataset = tf.data.Dataset.from_generator(
    lambda: generator(train_loader), # Use the generator function
    output_types=((tf.float32, tf.float32), tf.float32),
    output_shapes=(((batch_size, 256, 256, 3), (batch_size, 256, 256, 3)), (batch_size, 65536, 19))
)
val_dataset = tf.data.Dataset.from_generator(
    lambda: generator(val_loader), # Use the generator function
    output_types=((tf.float32, tf.float32), tf.float32),
    output_shapes=(((batch_size, 256, 256, 3), (batch_size, 256, 256, 3)), (batch_size, 65536, 19))
)

In [ ]:
# Train the model
epochs = 10
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

 Evaluate the model on the training and test dataset. The results must be shown as

- Print loss and accuracy of model for  test dataset.

- Predict semantically segmented images on 5 random example of test dataset.

- Visualize the 5 random examples alongside the ground truth and prediction.

In [ ]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(val_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
# Predict semantically segmented images on 5 random examples of the test dataset
num_samples = 5
random_indices = random.sample(range(len(rgb_val_paths)), num_samples)

In [ ]:
for i in random_indices:
    rgb_image = np.load(rgb_val_paths[i]) / 255.0
    depth_image = np.concatenate([np.load(depth_val_paths[i]), np.load(depth_val_paths[i]), np.load(depth_val_paths[i])], axis=-1) / np.max(np.load(depth_val_paths[i])) if np.max(np.load(depth_val_paths[i])) != 0 else np.concatenate([np.load(depth_val_paths[i]), np.load(depth_val_paths[i]), np.load(depth_val_paths[i])], axis=-1)
    ground_truth = np.load(label_val_paths[i])

    # Resize images to match model input shape
    rgb_image = cv2.resize(rgb_image, (256, 256))
    depth_image = cv2.resize(depth_image, (256, 256))

    # Make prediction
    prediction = model.predict([np.expand_dims(rgb_image, axis=0), np.expand_dims(depth_image, axis=0)])
    predicted_labels = np.argmax(prediction, axis=2).reshape((256, 256))

    # Visualize the results
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(rgb_image)
    plt.title("RGB Image")
    plt.subplot(1, 3, 2)
    plt.imshow(ground_truth)
    plt.title("Ground Truth")
    plt.subplot(1, 3, 3)
    plt.imshow(predicted_labels)
    plt.title("Predicted Labels")
    plt.show()

### Extra 5 points :

Implement FCNs for each sing modality and compare their accuracy with fusion model. I need the result of the following table in the same notebook.

# RESULT

| **Modality** | **Test Accuracy(%)** |
| -------- | -------- |
| RGB only |  |
| Depth Only |  |
| RBB and Depth Fusion |  |

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Model, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import Sequence, to_categorical
import numpy as np

# Assuming rgb_train_paths, depth_train_paths, label_train_paths,
# rgb_val_paths, depth_val_paths, label_val_paths are already defined

class UniqueResNet50(layers.Layer):
    def __init__(self, **kwargs):
        super(UniqueResNet50, self).__init__(**kwargs)
        self.resnet = ResNet50(weights='imagenet', include_top=False)
        for layer in self.resnet.layers:
            layer.trainable = False

    def call(self, inputs):
        return self.resnet(inputs)

def build_fcn_segmentation_model(input_shape=(256, 256, 3), num_classes=19):
    """Builds a Fully Convolutional Network (FCN) for image segmentation."""

    # Input layers
    input_rgb = layers.Input(shape=input_shape, name="input_rgb")
    input_depth = layers.Input(shape=input_shape, name="input_depth")

    # ResNet50 base models (pretrained) with unique instances
    rgb_stream = UniqueResNet50()(input_rgb)
    depth_stream = UniqueResNet50()(input_depth)

    # RGB stream
    rgb_stream = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(rgb_stream)
    rgb_stream = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(rgb_stream)
    rgb_stream = layers.Dropout(0.2)(rgb_stream)

    # Depth stream
    depth_stream = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(depth_stream)
    depth_stream = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(depth_stream)
    depth_stream = layers.Dropout(0.2)(depth_stream)

    # Concatenate streams
    concatenated = layers.Concatenate()([rgb_stream, depth_stream])

    # Transposed convolution (upsampling)
    upsampled = layers.Conv2DTranspose(num_classes, (64, 64), strides=(32, 32), padding='same')(concatenated)

    # Reshape (optional, for compatibility with some loss functions)
    reshaped = layers.Reshape((-1, num_classes))(upsampled)

    # Softmax activation
    output = layers.Activation('softmax')(reshaped)

    # Create the model
    model = Model(inputs=[input_rgb, input_depth], outputs=output)
    return model

class DataLoader(Sequence):
    def __init__(self, rgb_paths, depth_paths, label_paths, batch_size, num_classes=19):
        self.rgb_paths = rgb_paths
        self.depth_paths = depth_paths
        self.label_paths = label_paths
        self.batch_size = batch_size
        self.num_classes = num_classes

    def __len__(self):
        return int(np.ceil(len(self.rgb_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_rgb = self.rgb_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_depth = self.depth_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.label_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        return self.__generate_data(batch_rgb, batch_depth, batch_labels)

    def __generate_data(self, batch_rgb_files, batch_depth_files, batch_label_files):
        rgb_images = np.array([np.load(file) / 255.0 for file in batch_rgb_files])  # Normalize RGB
        depth_images = np.array([np.concatenate([np.load(file), np.load(file), np.load(file)], axis=-1) / np.max(np.load(file)) if np.max(np.load(file)) != 0 else np.concatenate([np.load(file), np.load(file), np.load(file)], axis=-1) for file in batch_depth_files])  # Normalize depth and duplicate channel
        labels = np.array([to_categorical(np.load(file), num_classes=self.num_classes) for file in batch_label_files])
        return [rgb_images, depth_images], labels

# Build the model
model = build_fcn_segmentation_model()

# Compile the model
optimizer = optimizers.SGD(learning_rate=0.01, decay=1e-5, momentum=0.9)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])

# Create data loaders
batch_size = 16 #adjust as needed.
train_loader = DataLoader(rgb_train_paths, depth_train_paths, label_train_paths, batch_size)
val_loader = DataLoader(rgb_val_paths, depth_val_paths, label_val_paths, batch_size)

# Train the model
epochs = 10
model.fit(train_loader, validation_data=val_loader, epochs=epochs)